This notebook contains all QIIME2 commands run for the Plastisphere metaanalysis paper (which were run separately for each study)
Here we have used the 2019.10 version of QIIME2, so these commands should all work with this version, but cannot be guarenteed with newer versions - although the changes between versions are not usually large
You can find details on installing this in a conda environment (as we have used) here:
https://docs.qiime2.org/2019.10/install/native/#install-qiime-2-within-a-conda-environment

Note that these commands were all run on a server. They may work on a local computer, but some steps may take too much memory. You should change the threads arguments (12 here) to an appropriate amount for your system.
We assume a basic understanding of these analyses and packages used here, but for further explanation on the steps involved see: https://github.com/LangilleLab/microbiome_helper/wiki/Amplicon-SOP-v2-(qiime2-2020.2)

1. Activate the QIIME2 environment

In [ ]:
import os
os.system('conda activate qiime2-2019.10')

2. Run quality checks on your data

In [ ]:
os.system('mkdir fastqc_out')
os.system('fastqc -t 12 raw_data/*.fastq.gz -o fastqc_out')
os.system('multiqc fastqc_out/')

You should now look at the multiqc_report.html (by copying to a local folder and opening it) to ensure that everything is high enough quality to proceed

3. Import your data to the QIIME2 format

In [ ]:
os.system('qiime tools import \
            --type SampleData[PairedEndSequencesWithQuality] \
            --input-path raw_data/ \
            --output-path reads.qza \
            --input-format CasavaOneEightSingleLanePerSampleDirFmt')

4. Optional - remove fastq files (these are no longer needed, so removing them saves space)

In [ ]:
os.system('rm -r raw_data/')

5. Trim primers (if present) with cutadapt. The primer sequences shown here are for 341F and 802R - these will need changing if you have used different primers

In [ ]:
os.system('qiime cutadapt trim-paired \
            --i-demultiplexed-sequences reads.qza \
            --p-cores 12 \
            --p-front-f CCTACGGGNGGCWGCAG \
            --p-front-r GACTACHVGGGTATCTAATCC \
            --p-discard-untrimmed \
            --p-no-indels \
            --o-trimmed-sequences reads_trimmed.qza')

6. Summarize trimmed fastq's (check that not too many reads have been removed by this step - if they have then the obvious first thing to check is that the primers haven't already been removed and the primer sequences have been typed correctly)

In [ ]:
os.system('qiime demux summarize \
            --i-data reads_trimmed.qza \
            --o-visualization reads_trimmed_summary.qzv')

7. Join paired ends (if the reads were already trimmed then just use reads.qza as the input here)

In [ ]:
os.system('qiime vsearch join-pairs \
            --i-demultiplexed-seqs reads_trimmed.qza \
            --o-joined-sequences reads_joined.qza')

8. Summarize the joined pairs (if too many reads were removed then you may need to play around with some of the other options at https://docs.qiime2.org/2020.2/plugins/available/vsearch/join-pairs/)

In [ ]:
os.system('qiime demux summarize \
            --i-data reads_joined.qza \
            --o-visualization reads_joined_summary.qzv')

9. Filter out low quality reads

In [ ]:
os.system('qiime quality-filter q-score-joined \
            --i-demux reads_joined.qza \
            --o-filter-stats filt_stats.qza \
            --o-filtered-sequences reads_joined_filtered.qza')

10. Summarize these reads (and look at where to trim)
You should look at the positions where the quality starts to drop below 30 and use these as trim lengths

In [ ]:
os.system('qiime demux summarize \
            --i-data reads_joined_filtered.qza \
            --o-visualization reads_joined_filtered_summary.qzv')

11. Run deblur (you can remove the --p-left-trim-len if you don't need to remove any from this end)

In [ ]:
os.system('qiime deblur denoise-16S \
            --i-demultiplexed-seqs reads_joined_filtered.qza \
            --p-trim-length 402 \
            --p-left-trim-len 0 \
            --p-sample-stats \
            --p-jobs-to-start 12 \
            --p-min-reads 1 \
            --output-dir deblur_output_quality')

12. Summarize the feature table to see how many reads we now have

In [ ]:
os.system('qiime feature-table summarize \
            --i-table deblur_output_quality/table.qza  \
            --o-visualization deblur_table_summary.qzv')